In [ ]:
# These are all the libraries that I have used from the beginning of the project to the end of the project.
# Some of them may be unnecessary.
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import os
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Loading the dateset
train_directory = '/kaggle/input/realistic-image/train'
validation_directory = '/kaggle/input/realistic-image/validation'
test_directory = '/kaggle/input/realistic-image/test'

train_labels_csv = '/kaggle/input/realistic-image/train.csv'
validation_labels_csv = '/kaggle/input/realistic-image/validation.csv'

train_labels_df = pd.read_csv(train_labels_csv)
validation_labels_df = pd.read_csv(validation_labels_csv)

def load_image_and_label(image_path, label, target_size):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img) / 255.0
    return img_array, label

def load_image(image_path, target_size):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img) / 255.0
    return img_array

def load_dataset(directory, labels_df, target_size):
    images = []
    labels = []
    for index, row in labels_df.iterrows():
        image_path = os.path.join(directory, f"{row['image_id']}.png")
        image, label = load_image_and_label(image_path, row['label'], target_size)
        images.append(image)
        labels.append(label)
    return np.array(images), np.array(labels)

def load_test_dataset(directory, target_size):
    images = []
    filenames = os.listdir(directory)
    image_ids = [x.split(".")[0] for x in filenames]
    for filename in filenames:
        image_path = os.path.join(directory, f"{filename}")
        image = load_image(image_path, target_size)
        images.append(image)
    return np.array(images), image_ids

target_size = (80, 80)

train_images, train_labels = load_dataset(train_directory, train_labels_df, target_size)
validation_images, validation_labels = load_dataset(validation_directory, validation_labels_df, target_size)
test_images, image_ids = load_test_dataset(test_directory, target_size)

train_validation_images = np.concatenate((train_images, validation_images))
train_validation_labels = np.concatenate((train_labels, validation_labels))

def create_model(input_shape=(80, 80, 3), num_classes=3):
    model = Sequential([
        Input(shape=input_shape),

        Conv2D(64, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(128, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        Conv2D(128, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(256, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        Conv2D(256, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Flatten(),

        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),

        Dense(num_classes, activation='softmax')
    ])
    return model

model = create_model()

learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=100, validation_data=(validation_images, validation_labels), callbacks=[reduce_lr, early_stopping])

# Functions to create training history and confusion matrix plots
def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'bo', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'ro', label='Training Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.title('Training and Validation ' + 'Loss')
    plt.legend()
    plt.show()

def plot_confusion_matrix(model, validation_images, validation_labels, num_classes):
    predictions = model.predict(validation_images)
    predicted_classes = np.argmax(predictions, axis=1)
    confusion_mtx = confusion_matrix(validation_labels, predicted_classes)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap="Reds")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

plot_training_history(history)
plot_confusion_matrix(model, validation_images, validation_labels, 3)

In [ ]:
Epoch 1/100
  3/329 ━━━━━━━━━━━━━━━━━━━━ 18s 58ms/step - accuracy: 0.2431 - loss: 2.5737 
W0000 00:00:1718373875.220099     120 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.4003 - loss: 1.5421
W0000 00:00:1718373898.407853     120 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1718373899.445418     118 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
329/329 ━━━━━━━━━━━━━━━━━━━━ 39s 79ms/step - accuracy: 0.4003 - loss: 1.5415 - val_accuracy: 0.3637 - val_loss: 1.2285 - learning_rate: 0.0010
Epoch 2/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 15s 46ms/step - accuracy: 0.5011 - loss: 1.0695 - val_accuracy: 0.4293 - val_loss: 1.3209 - learning_rate: 0.0010
Epoch 3/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 15s 46ms/step - accuracy: 0.6162 - loss: 0.8465 - val_accuracy: 0.6160 - val_loss: 0.8849 - learning_rate: 0.0010
Epoch 4/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 15s 46ms/step - accuracy: 0.6597 - loss: 0.7602 - val_accuracy: 0.6023 - val_loss: 0.8691 - learning_rate: 0.0010
Epoch 5/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 15s 45ms/step - accuracy: 0.6796 - loss: 0.7139 - val_accuracy: 0.4877 - val_loss: 1.5479 - learning_rate: 0.0010
Epoch 6/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 15s 45ms/step - accuracy: 0.6948 - loss: 0.6853 - val_accuracy: 0.5517 - val_loss: 1.2495 - learning_rate: 0.0010
Epoch 7/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 15s 45ms/step - accuracy: 0.7108 - loss: 0.6534 - val_accuracy: 0.5180 - val_loss: 1.5280 - learning_rate: 0.0010
Epoch 8/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 15s 45ms/step - accuracy: 0.7011 - loss: 0.6711 - val_accuracy: 0.6953 - val_loss: 0.6876 - learning_rate: 2.0000e-04
Epoch 9/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 15s 45ms/step - accuracy: 0.7484 - loss: 0.5776 - val_accuracy: 0.7337 - val_loss: 0.6001 - learning_rate: 2.0000e-04
Epoch 10/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 15s 45ms/step - accuracy: 0.7565 - loss: 0.5538 - val_accuracy: 0.7313 - val_loss: 0.5944 - learning_rate: 2.0000e-04
Epoch 11/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 15s 45ms/step - accuracy: 0.7707 - loss: 0.5260 - val_accuracy: 0.7260 - val_loss: 0.5962 - learning_rate: 2.0000e-04
Epoch 12/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 15s 45ms/step - accuracy: 0.7646 - loss: 0.5181 - val_accuracy: 0.7057 - val_loss: 0.7154 - learning_rate: 2.0000e-04
Epoch 13/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 15s 45ms/step - accuracy: 0.7768 - loss: 0.5029 - val_accuracy: 0.7420 - val_loss: 0.5771 - learning_rate: 2.0000e-04
...
Epoch 26/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 15s 45ms/step - accuracy: 0.8577 - loss: 0.3490 - val_accuracy: 0.7487 - val_loss: 0.5769 - learning_rate: 1.0000e-05
Epoch 27/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 15s 45ms/step - accuracy: 0.8487 - loss: 0.3529 - val_accuracy: 0.7547 - val_loss: 0.5792 - learning_rate: 1.0000e-05

![Accuracy Validation](accuracy_validation.png)

![Confusion Matrix](confusion_matrix.png)